# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Streaming processing final project** </center>
---
**Profesor**: Pablo Camarillo Ramirez
**Estudiante**: Santiago Montes de Oca

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://b064a789aeff:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dd6607e4-8436-46b8-a819-a47ca20d972b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

cd kafka<br/>
docker compose --file docker-compose.yml up -d<br/>

create a topic:<br/>

docker exec -it 31da5decad48 /opt/kafka/bin/kafka-topics.sh --create --topic raw-crawl-telemetry --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1

show that it exists:<br/>

docker exec -it 31da5decad48 /opt/kafka/bin/kafka-topics.sh --list --bootstrap-server localhost:9092




## Dataset and Stream creation (10 points)

In [2]:
topic = "raw-crawl-telemetry"

kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", topic) \
            .load()

kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



## Transformations and Actions (15 points)
In this section, you need to provide the code to clean and process data (create new columns, aggregate, run joins, etc.). In this section is NOT expected to have any action.

In [8]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructField, StringType
from santiagomdeo.spark_utils import SparkUtils

#process the received data:
#this is a single column string probably
crawler_telemetry_df = kafka_df.select(kafka_df.value.cast("string").alias("value_str"))


#format it with my library 
crawler_schema = SparkUtils.generate_schema([("from", "string"), ("to", "string"),("depth", "int")])

#god
crawler_extracted_df = crawler_telemetry_df.withColumn("telemetry", from_json(crawler_telemetry_df.value_str, crawler_schema))

crawler_extracted_df.printSchema()





string
string
int
root
 |-- value_str: string (nullable = true)
 |-- telemetry: struct (nullable = true)
 |    |-- from: string (nullable = true)
 |    |-- to: string (nullable = true)
 |    |-- depth: integer (nullable = true)



In [9]:
from pyspark.sql.functions import udf, count              #i made this "udf", so i might be importing something else
from santiagomdeo.utils3 import normalize_url

normalize_url_udf = udf(normalize_url, StringType())

In [10]:
parsed = crawler_extracted_df.select(
    col("telemetry.from").alias("from"),
    col("telemetry.to").alias("to"),
    col("telemetry.depth").alias("depth")
)

cleaned = (
    parsed
    .withColumn("from_url", normalize_url_udf(col("from")))
    .withColumn("to_url", normalize_url_udf(col("to")))
    .drop("from", "to")
)



In [11]:
aggregated = (
    cleaned
    .groupBy("from_url", "to_url", "depth")
    .agg(count("*").alias("appearances"))
)

cd to my lib
cd .\spark\notebooks\lib\santiagomdeo\producerStreaming\


### like the great gatsby get it? XD
docker build -t the-great-crawler .

### the actual container run
ctrl + lshift + v (allows to paste as one line)

also remeber the second you start this thing it fires a few messages so you have to have the outpur running

docker run -it
 --network=host
 -v "$(pwd)/verified_domains.txt:/app/verified_domains.txt"
 the-great-crawler
 https://www.iteso.mx
 2
 localhost:9092
 raw-crawl-telemetry

docker run -it --network=host -v $(pwd)/verified_domains.txt:/app/verified_domains.txt the-great-crawler https://www.iteso.mx 2 localhost:9092 raw-crawl-telemetry


In [ ]:
output = (
    aggregated.writeStream
              .format("console")
              .outputMode("update")     # <-- required for streaming aggregations
              .option("truncate", "false")
              .start()
)


25/11/23 19:37:38 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d2536e61-4470-47eb-99d6-73c0839f35c5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/23 19:37:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+------+-----+-----------+
|from_url|to_url|depth|appearances|
+--------+------+-----+-----------+
+--------+------+-----+-----------+



In [ ]:
output.awaitTermination(10)

False

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------------------+-----+-----------+
|from_url            |to_url              |depth|appearances|
+--------------------+--------------------+-----+-----------+
|https://www.iteso.mx|https://www.iteso.mx|0    |1          |
+--------------------+--------------------+-----+-----------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+----------------------------------------------------------------------------------------+-----+-----------+
|from_url            |to_url                                                                                  |depth|appearances|
+--------------------+----------------------------------------------------------------------------------------+-----+-----------+
|https://www.iteso.mx|https://vinculacion.iteso.mx                                                       

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----------------------------+-----+-----------+
|from_url            |to_url                       |depth|appearances|
+--------------------+-----------------------------+-----+-----------+
|https://www.iteso.mx|https://ite.so/magis508portal|0    |1          |
+--------------------+-----------------------------+-----+-----------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+----------------------------------------------------+-----+-----------+
|from_url            |to_url                                              |depth|appearances|
+--------------------+----------------------------------------------------+-----+-----------+
|https://www.iteso.mx|https://informedemedios.iteso.mx                    |0    |1          |
|https://www.iteso.mx|https://encartes.mx                                 |0    |1          |
|https://www.iteso.mx|https://analisisplural.iteso.mx/index.php/ap        |0    |1          |
|https://www.iteso.mx|https://sinectica.iteso.mx/index.php/SINECTICA/index|0    |1          |
|https://www.iteso.mx|https://publicaciones.iteso.mx                      |0    |1          |
|https://www.iteso.mx|https://complexus.iteso.mx                          |0    |1          |
+--------------------+-----------------------------------

# Persistence Data (15 points)

Once information is transformed, you need to document the process of persistence data in files using at least one vertical partition (partitionBy).  



In [ ]:
query_files = aggregated.writeStream \
    .trigger(processingTime="10 seconds") \
    .format("parquet") \
    .option("path", "/opt/spark/work-dir/data/crawler_clean_output/") \
    .option("checkpointLocation", "/opt/spark/work-dir/crawler_checkpoint") \
    .start()


# Power BI Dashboard (10 points)

With the files generated in the persistence section, you should create a Dashboard using Power BI to summarize and visualize the information you processed in the Data Pipeline

teach this is a note, the files generated are not exactly dashboard friendly and im not going to get any useful information from them, so the plan is to send to the dashboard health data.

power bi doesnt have a native kafka connector, so im going to receive it here, and send it in parquet files so that the dash board can see them, it will update every ten seconds or so, but i find this experiment much more interesting:

In [16]:
sc.stop()